## Excess Credit research at course level

In [0]:
import pandas as pd
import pickle
import glob
import numpy as np
from collections import Counter

import warnings
warnings.filterwarnings("ignore")

### 1. Read all needed Tables

In [2]:
# read table Awards
tblAwards = pd.read_excel("tblAwards.xlsx", index_col="PIDM").reset_index()
print(tblAwards.columns)
print(tblAwards.shape)
tblAwards.head(1)

Index(['PIDM', 'DownloadDte', 'CohortTrm', 'WCCID', 'A/C', 'Cons/SO',
       'TermsElpsd', 'TermsAtt', 'CumCrAtt', 'CumCrErn', 'CumCrTran',
       'CrAtt/Term', 'CrErn/Term', 'SEX', 'Eth', 'EthComputed', 'Citz',
       'BDate', 'Age', 'GradAge', 'DegC', 'Maj', 'CIP', 'Prog', 'Div',
       'ProgCredits', 'CreditGrp', 'CtlgYr', 'DegDte', 'GradAcadYr',
       'FirstDeg', 'Minor', 'Conc', 'FTrmCrAtt', 'Seq'],
      dtype='object')
(54304, 35)


,PIDM,DownloadDte,CohortTrm,WCCID,A/C,Cons/SO,TermsElpsd,TermsAtt,CumCrAtt,CumCrErn,CumCrTran,CrAtt/Term,CrErn/Term,SEX,Eth,EthComputed,Citz,BDate,Age,GradAge,DegC,Maj,CIP,Prog,Div,ProgCredits,CreditGrp,CtlgYr,DegDte,GradAcadYr,FirstDeg,Minor,Conc,FTrmCrAtt,Seq
0,1133,2019-09-09,199509,@00015315,A,SO,31.0,15.0,102.0,90.0,0.0,6.8,6.0,M,W,W,Y,1962-07-09,23.0,33.0,AGS,GST,240102.0,NaN,ATP,NaN,60-75,NaN,1995-12-16,1996.0,1,NaN,NaN,198509,11.10.10111.10.00.00.11.11.11.A


In [0]:
# only work on those entered after 200009
tblAwards = tblAwards[tblAwards['FTrmCrAtt'] >= 200009]

In [8]:
# read XGrade reference tables
XGrade = pd.read_excel("XGrade.xlsx")
print(XGrade.columns)
print(XGrade.shape)
XGrade.head(5)

Index(['Grade', 'GradeDesc', 'Succ'], dtype='object')
(43, 3)


,Grade,GradeDesc,Succ
0,*C,FORGIVE,NaN
1,*C-,FORGIVE,NaN
2,*C+,FORGIVE,NaN
3,*D,FORGIVE,NaN
4,*D-,FORGIVE,NaN


In [9]:
# read XGrade reference tables
XGrade_last = pd.read_excel("XGrade-last.xlsx")
print(XGrade_last.columns)
print(XGrade_last.shape)
XGrade_last.head(5)

Index(['Grade', 'GradeDesc', 'Succ'], dtype='object')
(43, 3)


,Grade,GradeDesc,Succ
0,*C,FORGIVE,NaN
1,*C-,FORGIVE,NaN
2,*C+,FORGIVE,NaN
3,*D,FORGIVE,NaN
4,*D-,FORGIVE,NaN


In [0]:
# read tblGrades
with open('tblGrades.txt') as f:
    tblGrades = f.readlines()
with open('tblGrades-last.txt') as f_last:
    tblGrades_last = f_last.readlines()

In [11]:
# read tblGrades table
names = ['Download', 'PIDM', 'Term', 'CRN', 'Level', 'Div', 'Dept',
       'Disc', 'Num', 'Sect', 'GradeX', 'Grade', 'Credit']
tblGrades = pd.read_csv('tblGrades.txt', names=names)
print(tblGrades.shape)
tblGrades.head(5)

(1737230, 13)


,Download,PIDM,Term,CRN,Level,Div,Dept,Disc,Num,Sect,GradeX,Grade,Credit
0,4/12/2001 0:00:00,1,199509,CN001,CR,BUS,CISD,CPS,185,NaN,AU,AU,4.0
1,4/12/2001 0:00:00,1,199509,CN002,CR,HSS,PERD,DAN,110,NaN,AU,AU,1.0
2,4/12/2001 0:00:00,1,199609,CN001,CR,BUS,CISD,CIS,225,NaN,AU,AU,2.0
3,4/12/2001 0:00:00,1,199609,CN002,CR,BUS,CISD,CPS,290,NaN,AU,AU,4.0
4,4/13/2001 0:00:00,1,199701,CN001,NC,CED,CEDD,BMG,CE52,NaN,NG,NG,0.0


In [12]:
# read last five years tblGrades table
names.append('Unnamed')
tblGrades_last = pd.read_csv('tblGrades-last.txt', names=names)
# kick out the last unnamed column
tblGrades_last = tblGrades_last.loc[:, ~tblGrades_last.columns.str.contains('^Unnamed')]

print(tblGrades_last.shape)
tblGrades_last.head(5)

(740710, 13)


,Download,PIDM,Term,CRN,Level,Div,Dept,Disc,Num,Sect,GradeX,Grade,Credit
0,5/9/2019 0:00:00,1,201801,10522,CR,HSS,PERD,YOG,101,05,W,W,2.0
1,9/17/2019 0:00:00,1,201809,92046,CR,HSS,PERD,YOG,101,01,AU,AU,2.0
2,10/20/2015 0:00:00,5,201201,10637,CR,HSS,HUMD,ART,114,01,A-,A-,4.0
3,10/20/2015 0:00:00,12,201009,94311,CR,HSS,FLGD,SPN,101,01,AU,AU,3.0
4,10/20/2015 0:00:00,12,201101,10135,CR,HSS,FLGD,SPN,111,02,AU,AU,5.0


In [0]:
mergeX_needed = ["PIDM", "RegTerm", "FTrmCrAtt", "SecTrmAtt", "SemCrAtt", "SemCrErn", "CumGPA", "PELL"]

In [29]:
chunk_size = 40000 #size of chunks relies on your available memory
separator = ","

# read in the data as chunks and save each chunk as pickle
reader = pd.read_csv("./tblMergeX.csv",sep=separator,chunksize=chunk_size, low_memory=False)
i=1
for chunk in reader:
    out_file = "./" + "data_{}.pkl".format(i)
    with open(out_file, "wb") as f:
        pickle.dump(chunk,f,pickle.HIGHEST_PROTOCOL)
    i+=1

# read in the pickles and append each pickle to your desired dataframe
pickle_path = "./" #Same Path as out_path i.e. where the pickle files are
data_p_files=[]
for name in glob.glob(pickle_path + "/data_*.pkl"):
    data_p_files.append(name)

tblMergeX = pd.DataFrame([])
for i in range(len(data_p_files)):
    tblMergeX = tblMergeX.append(pd.read_pickle(data_p_files[i]),ignore_index=True)
print(tblMergeX.shape)
tblMergeX.head(5)

(391284, 42)


,DownloadDte,UA,PIDM,RegTerm,BDate,Age,SEX,Eth,FTrmCrAtt,SecTrmAtt,SemCrAtt,SemCrErn,SemCrGPA,SemHonPts,SemGPA,EDLV,EGOL,Maj,Prog,CumCrAtt,CumCrErn,CumCrErnTrans,CumCrGPA,CumHonPts,CumGPA,CntTrmAtt,NextTrmAtt,NextYrAtt,PELL,SType,AdmtCd,AdmtTrm,Resid,ResdCd,RateCd,ALMT_MostRecent,ALRD_MostRecent,ALWR_MostRecent,ALMT_Initial,ALRD_Initial,ALWR_Initial,HSGPA
0,8/14/2019,0,891965,201705,3/3/1992,25.0,F,H,201209.0,0,11.0,11.0,11.0,41.9,3.809091,T4Y,AT,NURS,APNURS,54.0,49.0,18.0,41.0,149.6,3.648780,10.0,1.0,0.0,1,C,ST,201209.0,I,I,NaN,5.0,6.0,6.0,1.0,6.0,6.0,NaN
1,8/14/2019,0,891965,201709,3/3/1992,25.0,F,H,201209.0,0,10.0,10.0,6.0,23.1,3.850000,T4Y,AT,NURS,APNURS,65.0,60.0,18.0,52.0,191.5,3.682692,11.0,1.0,0.0,1,C,ST,201209.0,I,I,NaN,5.0,6.0,6.0,1.0,6.0,6.0,NaN
2,8/14/2019,0,891965,201801,3/3/1992,25.0,F,H,201209.0,0,11.0,11.0,6.0,22.8,3.800000,T4Y,AT,NURS,APNURS,75.0,70.0,18.0,58.0,214.6,3.700000,12.0,0.0,0.0,1,C,ST,201209.0,I,I,NaN,5.0,6.0,6.0,1.0,6.0,6.0,NaN
3,8/14/2019,0,891973,201205,5/21/1992,19.0,F,W,201205.0,1,7.0,7.0,7.0,26.3,3.757143,TCC,AS,ADA,CTADA,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0,T,ST,201205.0,O,O,NaN,3.0,6.0,6.0,3.0,6.0,6.0,NaN
4,8/14/2019,0,891973,201209,5/21/1992,20.0,F,W,201205.0,1,9.0,9.0,9.0,36.0,4.000000,TCC,AS,ADA,CTADA,7.0,7.0,28.0,7.0,26.3,3.757143,2.0,1.0,0.0,0,C,ST,201205.0,O,O,NaN,3.0,6.0,6.0,3.0,6.0,6.0,NaN


In [18]:
min(tblMergeX['RegTerm'])

200809

In [14]:
# read in the data as chunks and save each chunk as pickle
reader = pd.read_csv("./tblMergeX_older.csv",sep=separator,chunksize=chunk_size, low_memory=False)
i=1
for chunk in reader:
    out_file = "./" + "dataOlder_{}.pkl".format(i)
    with open(out_file, "wb") as f:
        pickle.dump(chunk,f,pickle.HIGHEST_PROTOCOL)
    i+=1

# read in the pickles and append each pickle to your desired dataframe
pickle_path = "./" #Same Path as out_path i.e. where the pickle files are
data_p_files=[]
for name in glob.glob(pickle_path + "/dataOlder_*.pkl"):
    data_p_files.append(name)

tblMergeX_older = pd.DataFrame([])
for i in range(len(data_p_files)):
    tblMergeX_older = tblMergeX_older.append(pd.read_pickle(data_p_files[i]),ignore_index=True)
print(tblMergeX_older.shape)
tblMergeX_older.head(5)

(701085, 41)


,DownloadDte,UA,PIDM,RegTerm,BDate,Age,SEX,Eth,FTrmCrAtt,SecTrmAtt,SemCrAtt,SemCrErn,SemCrGPA,SemHonPts,SemGPA,EDLV,EGOL,Maj,Prog,CumCrAtt,CumCrErn,CumCrErnTrans,CumCrGPA,CumHonPts,CumGPA,CntTrmAtt,NextTrmAtt,NextYrAtt,PELL,SType,AdmtCd,AdmtTrm,Resid,ResdCd,RateCd,ALMT_Initial,ALRD_Initial,ALWR_Initial,ALMT_MostRecent,ALRD_MostRecent,ALWR_MostRecent
0,11/16/2018,0,718539,200509,6/6/1976,29.0,F,A,200509.0,1,4.0,4.0,4.0,14.8,3.7,CGD,CE,BPHO,CTBPHO,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0,R,ST,200509.0,I,I,NaN,NaN,NaN,NaN,0.0,6.0,6.0
1,11/16/2018,0,718539,200601,6/6/1976,29.0,F,A,200509.0,1,3.0,3.0,3.0,11.1,3.7,CGD,CE,BPHO,CTBPHO,4.0,4.0,NaN,4.0,14.8,3.700000,2.0,1.0,1.0,0,R,ST,200509.0,I,I,NaN,NaN,NaN,NaN,0.0,6.0,6.0
2,11/16/2018,0,718539,200609,6/6/1976,30.0,F,A,200509.0,1,4.0,4.0,4.0,13.2,3.3,CGD,CE,BPHO,CTBPHO,7.0,7.0,NaN,7.0,25.9,3.700000,3.0,1.0,1.0,0,R,ST,200509.0,I,I,NaN,NaN,NaN,NaN,0.0,6.0,6.0
3,11/16/2018,0,718539,200701,6/6/1976,30.0,F,A,200509.0,1,7.0,7.0,7.0,28.0,4.0,CGD,CE,BPHO,CTBPHO,11.0,11.0,NaN,11.0,39.1,3.554545,4.0,1.0,0.0,0,R,ST,200509.0,I,I,NaN,NaN,NaN,NaN,0.0,6.0,6.0
4,11/16/2018,0,718539,200709,6/6/1976,31.0,F,A,200509.0,1,3.0,3.0,3.0,12.0,4.0,CGD,CE,BPHO,CTBPHO,18.0,18.0,NaN,18.0,67.1,3.727778,5.0,0.0,0.0,0,C,ST,200509.0,I,I,NaN,NaN,NaN,NaN,0.0,6.0,6.0


In [30]:
tblMergeX_older = tblMergeX_older[(tblMergeX_older['RegTerm'] >= 200009) & (tblMergeX_older['RegTerm'] <= 200805)]
tblMergeX_combine = tblMergeX[mergeX_needed].append(tblMergeX_older[mergeX_needed])
print(tblMergeX_combine.shape)
tblMergeX_combine.head(5)

(651694, 8)


,PIDM,RegTerm,FTrmCrAtt,SecTrmAtt,SemCrAtt,SemCrErn,CumGPA,PELL
0,891965,201705,201209.0,0,11.0,11.0,3.648780,1
1,891965,201709,201209.0,0,10.0,10.0,3.682692,1
2,891965,201801,201209.0,0,11.0,11.0,3.700000,1
3,891973,201205,201205.0,1,7.0,7.0,NaN,0
4,891973,201209,201205.0,1,9.0,9.0,3.757143,0


### 2. Only keep latest 10 years records in all tables and combine

In [31]:
print(tblAwards.shape)

(41326, 35)


In [32]:
ID_list = tblAwards['PIDM'].tolist()
tblGrades = tblGrades[tblGrades['PIDM'].isin(ID_list)]
tblGrades = tblGrades.append(tblGrades_last)
print(tblGrades.shape)

(1513002, 13)


In [33]:
dict(Counter(tblGrades['Level']))

{'CE': 12327, 'CR': 1485963, 'NC': 14712}

In [34]:
tblMergeX_combine = tblMergeX_combine[tblMergeX_combine['PIDM'].isin(ID_list)]
print(tblMergeX_combine.shape)

(205924, 8)


### 3. Sets merging and calculation

In [35]:
# Only working on those with associate degrees
tblAwards = tblAwards[tblAwards['A/C'] == 'A']
print(tblAwards.shape)

(15774, 35)


In [0]:
# restore all needed info in merging set
merging = tblAwards.copy()
merging = merging[merging['FTrmCrAtt'] < 999999]
# delete useless columns
del merging['DownloadDte']
del merging['WCCID']
del merging['CIP']
del merging['Conc']
del merging['Seq']

ID_list = merging['PIDM'].tolist()

In [37]:
merging = merging.reset_index(drop=True)
merging.head(1)

,PIDM,CohortTrm,A/C,Cons/SO,TermsElpsd,TermsAtt,CumCrAtt,CumCrErn,CumCrTran,CrAtt/Term,CrErn/Term,SEX,Eth,EthComputed,Citz,BDate,Age,GradAge,DegC,Maj,Prog,Div,ProgCredits,CreditGrp,CtlgYr,DegDte,GradAcadYr,FirstDeg,Minor,FTrmCrAtt
0,532954,200101,A,Cons,2.0,2.0,28.0,73.0,45.0,14.0,14.0,F,W,W,Y,1979-05-03,21.0,21.0,AAS,PHOT,APPHOT,BCT,61.0,60-75,2000.0,2001-04-30,2001.0,1,NaN,200009


In [38]:
passSet = XGrade[XGrade['Succ'] == 1]['Grade'].tolist()
print(passSet)
# create a semester time line
print(min(merging['FTrmCrAtt']))
print(max(merging['FTrmCrAtt']))
time_line = [min(merging['FTrmCrAtt'])]

year = min(merging['FTrmCrAtt'])
while year != max(merging['FTrmCrAtt']):
  if year%10 in (1, 5):
    year = year + 4
  else: # year ends at 9
    year = year + 92
  time_line.append(year)
print(time_line)
time_line_len = len(time_line)
print(time_line[-1])

['A', 'A-', 'B', 'B-', 'B+', 'C', 'C-', 'C+', 'P', 'S']
200009
201905
[200009, 200101, 200105, 200109, 200201, 200205, 200209, 200301, 200305, 200309, 200401, 200405, 200409, 200501, 200505, 200509, 200601, 200605, 200609, 200701, 200705, 200709, 200801, 200805, 200809, 200901, 200905, 200909, 201001, 201005, 201009, 201101, 201105, 201109, 201201, 201205, 201209, 201301, 201305, 201309, 201401, 201405, 201409, 201501, 201505, 201509, 201601, 201605, 201609, 201701, 201705, 201709, 201801, 201805, 201809, 201901, 201905]
201905


In [0]:
merging['TotalNumCourses'] = np.nan
merging['RepeatedNumCourses'] = np.nan
merging['NumCE'] = np.nan
merging['NumCR'] = np.nan
merging['NumNC'] = np.nan
merging['SuccessNum'] = np.nan
merging['NotSuccessNum'] = np.nan
merging['<100'] = np.nan
merging['100s'] = np.nan
merging['200s'] = np.nan
merging['300s'] = np.nan
merging['>1000'] = np.nan
merging['FYrCreditsAttained'] = np.nan
merging['FYrCreditsEarned'] = np.nan

for idx in range(len(merging)):
  pidm = merging['PIDM'][idx]
  # information from tblGrades
  temp = tblGrades[tblGrades['PIDM'] == pidm]
  temp['CourseInfo'] = temp.Disc.str.cat(temp.Num)
  temp = temp.drop_duplicates(subset=['CourseInfo', 'GradeX'], keep='first')
  # filter courses during degree period
  FTrmCrAtt = merging['FTrmCrAtt'][idx]
  CohortTrm = merging['CohortTrm'][idx]
  temp = temp[(temp['Term'] >= FTrmCrAtt) & (temp['Term'] <= CohortTrm)]

  merging['TotalNumCourses'][idx] = len(temp) # total number of courses
  course_list = temp['CourseInfo'].tolist()
  merging['RepeatedNumCourses'][idx] = len(set([x for x in course_list if course_list.count(x) > 1])) # repeated courses

  merging['NumCE'][idx] = len(temp[temp['Level'] == "CE"]) # CE courses
  merging['NumCR'][idx] = len(temp[temp['Level'] == "CR"]) # CR courses
  merging['NumNC'][idx] = len(temp[temp['Level'] == "NC"]) # NC courses
  # add a SucOrNot column
  temp['SucOrNot'] = 0
  temp.SucOrNot[temp['GradeX'].isin(passSet)] = 1
  merging['SuccessNum'][idx] = len(temp[temp['SucOrNot'] == 1]["CourseInfo"].unique()) # success courses
  merging['NotSuccessNum'][idx] = len(temp['CourseInfo'].unique()) - merging['SuccessNum'][idx] # not sucess courses

  # count courses levels
  temp['Num'] = temp['Num'].str.replace(r'\D', '')
  temp = temp[temp['Num'] != ""]
  temp['Num'] = temp['Num'].astype(int)
  merging['<100'][idx] = len(temp[temp['Num'] < 100])
  merging['100s'][idx] = len(temp[(temp['Num'] >= 100) & (temp['Num'] < 200)])
  merging['200s'][idx] = len(temp[(temp['Num'] >= 200) & (temp['Num'] < 300)])
  merging['300s'][idx] = len(temp[(temp['Num'] >= 300) & (temp['Num'] < 1000)])
  merging['>1000'][idx] = len(temp[temp['Num'] >= 1000])

  # count credits earned and attained in first year 
  # use tblMergeX_combine to count 
  location = time_line.index(FTrmCrAtt)
  if location >= time_line_len - 2:
    # last one - shorter than one year
    limit = time_line[location:]
  else:
    limit = time_line[location: (location + 3)]
  temp_merge = tblMergeX_combine[(tblMergeX_combine['PIDM'] == pidm) & (tblMergeX_combine['RegTerm'].isin(limit))]
  if len(temp_merge) > 0:
    merging['FYrCreditsEarned'][idx] = sum(temp_merge['SemCrErn'])
    merging['FYrCreditsAttained'][idx] = sum(temp_merge['SemCrAtt'])

  if idx %200 == 0:
    print(idx)

In [43]:
merging.head(10)

,PIDM,CohortTrm,A/C,Cons/SO,TermsElpsd,TermsAtt,CumCrAtt,CumCrErn,CumCrTran,CrAtt/Term,CrErn/Term,SEX,Eth,EthComputed,Citz,BDate,Age,GradAge,DegC,Maj,Prog,Div,ProgCredits,CreditGrp,CtlgYr,DegDte,GradAcadYr,FirstDeg,Minor,FTrmCrAtt,TotalNumCourses,RepeatedNumCourses,NumCE,NumCR,NumNC,SuccessNum,NotSuccessNum,<100,100s,200s,300s,>1000,FYrCreditsAttained,FYrCreditsEarned
0,532954,200101,A,Cons,2.0,2.0,28.0,73.0,45.0,14.000000,14.000000,F,W,W,Y,1979-05-03,21.0,21.0,AAS,PHOT,APPHOT,BCT,61.0,60-75,2000.0,2001-04-30,2001.0,1,NaN,200009,9.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,4.0,5.0,0.0,0.0,28.0,28.0
1,544910,200101,A,Cons,2.0,2.0,24.0,68.0,44.0,12.000000,12.000000,M,W,W,F,1976-11-22,23.0,24.0,AAS,GSAS,APGSAS,MSE,60.0,60-75,2000.0,2001-04-30,2001.0,1,NaN,200009,9.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,8.0,1.0,0.0,0.0,24.0,24.0
2,549042,200101,A,Cons,2.0,2.0,30.0,60.0,30.0,15.000000,15.000000,M,H,H,F,1973-01-28,27.0,28.0,AAS,GSAS,APGSAS,MSE,60.0,60-75,2000.0,2001-04-30,2001.0,1,NaN,200009,11.0,0.0,0.0,11.0,0.0,11.0,0.0,0.0,10.0,1.0,0.0,0.0,34.0,34.0
3,558488,200101,A,Cons,1.0,1.0,16.0,16.0,0.0,16.000000,16.000000,F,W,W,F,1978-01-05,22.0,23.0,AA,GSLA,AAGSLA,HSS,60.0,60-75,2000.0,2001-04-30,2001.0,1,NaN,200101,5.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,16.0,16.0
4,198722,200105,A,Cons,3.0,3.0,28.0,67.0,39.0,9.333333,9.333333,M,W,W,Y,1968-08-28,32.0,32.0,AAS,GSAS,APGSAS,MSE,60.0,60-75,2000.0,2001-08-22,2001.0,1,NaN,200009,9.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,8.0,1.0,0.0,0.0,28.0,28.0
5,536784,200105,A,Cons,3.0,3.0,45.0,90.0,45.0,15.000000,15.000000,F,W,W,Y,1954-08-13,46.0,47.0,AAS,GSAS,APGSAS,MSE,60.0,60-75,2000.0,2001-08-22,2001.0,1,NaN,200009,13.0,0.0,0.0,13.0,0.0,13.0,0.0,0.0,10.0,3.0,0.0,0.0,45.0,45.0
6,542193,200105,A,Cons,3.0,3.0,37.0,65.0,31.0,12.333333,11.333333,M,W,W,Y,1972-02-01,28.0,29.0,AAS,GSAS,APGSAS,MSE,60.0,60-75,2000.0,2001-08-22,2001.0,1,NaN,200009,12.0,0.0,0.0,12.0,0.0,11.0,1.0,0.0,9.0,3.0,0.0,0.0,37.0,34.0
7,543475,200105,A,Cons,3.0,3.0,35.0,66.0,31.0,11.666667,11.666667,F,W,W,Y,1969-04-03,31.0,32.0,AAS,DAC,APDAC,HLT,38.0,60-75,2000.0,2001-08-22,2001.0,1,NaN,200009,17.0,0.0,0.0,17.0,0.0,17.0,0.0,0.0,14.0,3.0,0.0,0.0,35.0,35.0
8,543541,200105,A,Cons,3.0,3.0,30.0,75.0,48.0,10.000000,9.000000,F,W,W,Y,1978-04-14,22.0,23.0,AAS,RAD,RAD,HLT,70.0,60-75,1999.0,2001-06-28,2001.0,1,NaN,200009,11.0,1.0,0.0,11.0,0.0,10.0,0.0,0.0,3.0,8.0,0.0,0.0,30.0,27.0
9,513135,200109,A,Cons,4.0,2.0,17.0,63.0,46.0,8.500000,8.500000,M,W,W,Y,1973-01-24,27.0,28.0,AAS,GSAS,APGSAS,MSE,60.0,60-75,2000.0,2001-12-21,2002.0,1,NaN,200009,5.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,2.0,3.0,0.0,0.0,17.0,17.0


In [45]:
basicInfo = ['PIDM', 'Age', 'SEX', 'Eth']
majorInfo = ['Maj', 'Prog']

GPAInfo = ['SemGPA', 'CumGPA']
termInfo = ['RegTerm', 'FTrmCrAtt', 'CntTrmAtt']
creditInfo = ['CumCrAtt', 'CumCrErn', 'CumCrErnTrans']

TotalInfo = basicInfo + majorInfo + GPAInfo + termInfo + creditInfo
# sub_tblMergeX - contain all needed info at this point
sub_tblMergeX = tblMergeX[TotalInfo].append(tblMergeX_older[TotalInfo])
print(sub_tblMergeX.shape)
sub_tblMergeX.head(5)

(651694, 14)


,PIDM,Age,SEX,Eth,Maj,Prog,SemGPA,CumGPA,RegTerm,FTrmCrAtt,CntTrmAtt,CumCrAtt,CumCrErn,CumCrErnTrans
0,891965,25.0,F,H,NURS,APNURS,3.809091,3.648780,201705,201209.0,10.0,54.0,49.0,18.0
1,891965,25.0,F,H,NURS,APNURS,3.850000,3.682692,201709,201209.0,11.0,65.0,60.0,18.0
2,891965,25.0,F,H,NURS,APNURS,3.800000,3.700000,201801,201209.0,12.0,75.0,70.0,18.0
3,891973,19.0,F,W,ADA,CTADA,3.757143,NaN,201205,201205.0,1.0,NaN,NaN,NaN
4,891973,20.0,F,W,ADA,CTADA,4.000000,3.757143,201209,201205.0,2.0,7.0,7.0,28.0


### 4. Merge program required credtis and excess credits 

In [46]:
# read table XPrograms
XPrograms = pd.read_excel("XPrograms.xlsx", index_col='ID').reset_index()
print(XPrograms.columns)
print(XPrograms.shape)
XPrograms.head(1)

Index(['ID', 'Maj', 'MajName', 'ProgName', 'Prog', 'Deg', 'Div', 'Dept', 'LYr',
       'FYr', 'ProgCredits', 'Bulletin Year', 'Banner Year', 'Extra'],
      dtype='object')
(3517, 14)


,ID,Maj,MajName,ProgName,Prog,Deg,Div,Dept,LYr,FYr,ProgCredits,Bulletin Year,Banner Year,Extra
0,3448,01A1,3+1 Graphic Design- EMU (X),3+1 Graphic Design- EMU (X),TR01A1,TR,BCT,DMAD,2015.0,2013.0,94.0,2013,2014,NaN


In [47]:
print(merging.shape)
merging = pd.merge(merging, XPrograms[['Maj', 'Bulletin Year', 'ProgCredits']], how='left',
                   left_on=['Maj', 'GradAcadYr'], right_on=['Maj', 'Bulletin Year'])
# set new column for credits earned from WCC
merging['CumCrEarnedWCC'] = merging['CumCrErn'] - merging['CumCrTran']
print(merging.shape)
merging.head(5)

(15773, 44)
(15773, 47)


,PIDM,CohortTrm,A/C,Cons/SO,TermsElpsd,TermsAtt,CumCrAtt,CumCrErn,CumCrTran,CrAtt/Term,CrErn/Term,SEX,Eth,EthComputed,Citz,BDate,Age,GradAge,DegC,Maj,Prog,Div,ProgCredits_x,CreditGrp,CtlgYr,DegDte,GradAcadYr,FirstDeg,Minor,FTrmCrAtt,TotalNumCourses,RepeatedNumCourses,NumCE,NumCR,NumNC,SuccessNum,NotSuccessNum,<100,100s,200s,300s,>1000,FYrCreditsAttained,FYrCreditsEarned,Bulletin Year,ProgCredits_y,CumCrEarnedWCC
0,532954,200101,A,Cons,2.0,2.0,28.0,73.0,45.0,14.000000,14.000000,F,W,W,Y,1979-05-03,21.0,21.0,AAS,PHOT,APPHOT,BCT,61.0,60-75,2000.0,2001-04-30,2001.0,1,NaN,200009,9.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,4.0,5.0,0.0,0.0,28.0,28.0,2001.0,60.0,28.0
1,544910,200101,A,Cons,2.0,2.0,24.0,68.0,44.0,12.000000,12.000000,M,W,W,F,1976-11-22,23.0,24.0,AAS,GSAS,APGSAS,MSE,60.0,60-75,2000.0,2001-04-30,2001.0,1,NaN,200009,9.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,8.0,1.0,0.0,0.0,24.0,24.0,2001.0,60.0,24.0
2,549042,200101,A,Cons,2.0,2.0,30.0,60.0,30.0,15.000000,15.000000,M,H,H,F,1973-01-28,27.0,28.0,AAS,GSAS,APGSAS,MSE,60.0,60-75,2000.0,2001-04-30,2001.0,1,NaN,200009,11.0,0.0,0.0,11.0,0.0,11.0,0.0,0.0,10.0,1.0,0.0,0.0,34.0,34.0,2001.0,60.0,30.0
3,558488,200101,A,Cons,1.0,1.0,16.0,16.0,0.0,16.000000,16.000000,F,W,W,F,1978-01-05,22.0,23.0,AA,GSLA,AAGSLA,HSS,60.0,60-75,2000.0,2001-04-30,2001.0,1,NaN,200101,5.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,16.0,16.0,2001.0,60.0,16.0
4,198722,200105,A,Cons,3.0,3.0,28.0,67.0,39.0,9.333333,9.333333,M,W,W,Y,1968-08-28,32.0,32.0,AAS,GSAS,APGSAS,MSE,60.0,60-75,2000.0,2001-08-22,2001.0,1,NaN,200009,9.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,8.0,1.0,0.0,0.0,28.0,28.0,2001.0,60.0,28.0


In [48]:
merging['ProgCredits_x'].fillna(merging['ProgCredits_y'], inplace=True)
del merging['ProgCredits_y']
print("Number of not available program credits found: ", len(merging[merging['ProgCredits_x'].isna()]))
# play with excess credits
merging['ExcessCr'] = merging['CumCrErn'] - merging['ProgCredits_x']

Number of not available program credits found:  110


In [49]:
merging.head(5)

,PIDM,CohortTrm,A/C,Cons/SO,TermsElpsd,TermsAtt,CumCrAtt,CumCrErn,CumCrTran,CrAtt/Term,CrErn/Term,SEX,Eth,EthComputed,Citz,BDate,Age,GradAge,DegC,Maj,Prog,Div,ProgCredits_x,CreditGrp,CtlgYr,DegDte,GradAcadYr,FirstDeg,Minor,FTrmCrAtt,TotalNumCourses,RepeatedNumCourses,NumCE,NumCR,NumNC,SuccessNum,NotSuccessNum,<100,100s,200s,300s,>1000,FYrCreditsAttained,FYrCreditsEarned,Bulletin Year,CumCrEarnedWCC,ExcessCr
0,532954,200101,A,Cons,2.0,2.0,28.0,73.0,45.0,14.000000,14.000000,F,W,W,Y,1979-05-03,21.0,21.0,AAS,PHOT,APPHOT,BCT,61.0,60-75,2000.0,2001-04-30,2001.0,1,NaN,200009,9.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,4.0,5.0,0.0,0.0,28.0,28.0,2001.0,28.0,12.0
1,544910,200101,A,Cons,2.0,2.0,24.0,68.0,44.0,12.000000,12.000000,M,W,W,F,1976-11-22,23.0,24.0,AAS,GSAS,APGSAS,MSE,60.0,60-75,2000.0,2001-04-30,2001.0,1,NaN,200009,9.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,8.0,1.0,0.0,0.0,24.0,24.0,2001.0,24.0,8.0
2,549042,200101,A,Cons,2.0,2.0,30.0,60.0,30.0,15.000000,15.000000,M,H,H,F,1973-01-28,27.0,28.0,AAS,GSAS,APGSAS,MSE,60.0,60-75,2000.0,2001-04-30,2001.0,1,NaN,200009,11.0,0.0,0.0,11.0,0.0,11.0,0.0,0.0,10.0,1.0,0.0,0.0,34.0,34.0,2001.0,30.0,0.0
3,558488,200101,A,Cons,1.0,1.0,16.0,16.0,0.0,16.000000,16.000000,F,W,W,F,1978-01-05,22.0,23.0,AA,GSLA,AAGSLA,HSS,60.0,60-75,2000.0,2001-04-30,2001.0,1,NaN,200101,5.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,16.0,16.0,2001.0,16.0,-44.0
4,198722,200105,A,Cons,3.0,3.0,28.0,67.0,39.0,9.333333,9.333333,M,W,W,Y,1968-08-28,32.0,32.0,AAS,GSAS,APGSAS,MSE,60.0,60-75,2000.0,2001-08-22,2001.0,1,NaN,200009,9.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,8.0,1.0,0.0,0.0,28.0,28.0,2001.0,28.0,7.0


#### 5. GPA and PELL information merging

In [50]:
PELL_table = tblMergeX_combine[['PIDM', 'PELL']].groupby('PIDM').sum().reset_index()
PELL_table.loc[PELL_table['PELL'] >= 1, 'PELL'] = 1
print(PELL_table['PELL'].unique())

print(PELL_table.head(5))
GPA_table = tblMergeX_combine[['PIDM', 'RegTerm', 'CumGPA']]
GPA_table = GPA_table.sort_values(by=['PIDM', 'RegTerm'])
GPA_table = GPA_table.drop_duplicates(subset=['PIDM'], keep='last')
print(GPA_table.head(5))

[0 1]
   PIDM  PELL
0   454     0
1  1210     0
2  1468     1
3  1545     0
4  1662     0
        PIDM  RegTerm    CumGPA
200274   454   201905  3.758333
361048  1210   200601  3.400000
200420  1468   201609  3.594203
200438  1545   201401  3.786957
200458  1662   201901  4.000000


In [51]:
result = pd.merge(merging, PELL_table, how='left', left_on=['PIDM'], right_on=['PIDM'])
print(result.shape)
print(len(result[result['PELL'].isna()]))
result = pd.merge(result, GPA_table[['PIDM', 'CumGPA']], how='left', left_on=['PIDM'], right_on=['PIDM'])
print(result.shape)
print(len(result[result['CumGPA'].isna()]))
result.head(10)

(15773, 48)
0
(15773, 49)
33


,PIDM,CohortTrm,A/C,Cons/SO,TermsElpsd,TermsAtt,CumCrAtt,CumCrErn,CumCrTran,CrAtt/Term,CrErn/Term,SEX,Eth,EthComputed,Citz,BDate,Age,GradAge,DegC,Maj,Prog,Div,ProgCredits_x,CreditGrp,CtlgYr,DegDte,GradAcadYr,FirstDeg,Minor,FTrmCrAtt,TotalNumCourses,RepeatedNumCourses,NumCE,NumCR,NumNC,SuccessNum,NotSuccessNum,<100,100s,200s,300s,>1000,FYrCreditsAttained,FYrCreditsEarned,Bulletin Year,CumCrEarnedWCC,ExcessCr,PELL,CumGPA
0,532954,200101,A,Cons,2.0,2.0,28.0,73.0,45.0,14.000000,14.000000,F,W,W,Y,1979-05-03,21.0,21.0,AAS,PHOT,APPHOT,BCT,61.0,60-75,2000.0,2001-04-30,2001.0,1,NaN,200009,9.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,4.0,5.0,0.0,0.0,28.0,28.0,2001.0,28.0,12.0,0,3.292857
1,544910,200101,A,Cons,2.0,2.0,24.0,68.0,44.0,12.000000,12.000000,M,W,W,F,1976-11-22,23.0,24.0,AAS,GSAS,APGSAS,MSE,60.0,60-75,2000.0,2001-04-30,2001.0,1,NaN,200009,9.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,8.0,1.0,0.0,0.0,24.0,24.0,2001.0,24.0,8.0,0,3.383333
2,549042,200101,A,Cons,2.0,2.0,30.0,60.0,30.0,15.000000,15.000000,M,H,H,F,1973-01-28,27.0,28.0,AAS,GSAS,APGSAS,MSE,60.0,60-75,2000.0,2001-04-30,2001.0,1,NaN,200009,11.0,0.0,0.0,11.0,0.0,11.0,0.0,0.0,10.0,1.0,0.0,0.0,34.0,34.0,2001.0,30.0,0.0,0,3.473333
3,558488,200101,A,Cons,1.0,1.0,16.0,16.0,0.0,16.000000,16.000000,F,W,W,F,1978-01-05,22.0,23.0,AA,GSLA,AAGSLA,HSS,60.0,60-75,2000.0,2001-04-30,2001.0,1,NaN,200101,5.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,16.0,16.0,2001.0,16.0,-44.0,0,3.943750
4,198722,200105,A,Cons,3.0,3.0,28.0,67.0,39.0,9.333333,9.333333,M,W,W,Y,1968-08-28,32.0,32.0,AAS,GSAS,APGSAS,MSE,60.0,60-75,2000.0,2001-08-22,2001.0,1,NaN,200009,9.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,8.0,1.0,0.0,0.0,28.0,28.0,2001.0,28.0,7.0,0,3.362500
5,536784,200105,A,Cons,3.0,3.0,45.0,90.0,45.0,15.000000,15.000000,F,W,W,Y,1954-08-13,46.0,47.0,AAS,GSAS,APGSAS,MSE,60.0,60-75,2000.0,2001-08-22,2001.0,1,NaN,200009,13.0,0.0,0.0,13.0,0.0,13.0,0.0,0.0,10.0,3.0,0.0,0.0,45.0,45.0,2001.0,45.0,30.0,0,3.966667
6,542193,200105,A,Cons,3.0,3.0,37.0,65.0,31.0,12.333333,11.333333,M,W,W,Y,1972-02-01,28.0,29.0,AAS,GSAS,APGSAS,MSE,60.0,60-75,2000.0,2001-08-22,2001.0,1,NaN,200009,12.0,0.0,0.0,12.0,0.0,11.0,1.0,0.0,9.0,3.0,0.0,0.0,37.0,34.0,2001.0,34.0,5.0,0,3.777778
7,543475,200105,A,Cons,3.0,3.0,35.0,66.0,31.0,11.666667,11.666667,F,W,W,Y,1969-04-03,31.0,32.0,AAS,DAC,APDAC,HLT,38.0,60-75,2000.0,2001-08-22,2001.0,1,NaN,200009,17.0,0.0,0.0,17.0,0.0,17.0,0.0,0.0,14.0,3.0,0.0,0.0,35.0,35.0,2001.0,35.0,28.0,0,3.958333
8,543541,200105,A,Cons,3.0,3.0,30.0,75.0,48.0,10.000000,9.000000,F,W,W,Y,1978-04-14,22.0,23.0,AAS,RAD,RAD,HLT,70.0,60-75,1999.0,2001-06-28,2001.0,1,NaN,200009,11.0,1.0,0.0,11.0,0.0,10.0,0.0,0.0,3.0,8.0,0.0,0.0,30.0,27.0,2001.0,27.0,5.0,0,2.412500
9,513135,200109,A,Cons,4.0,2.0,17.0,63.0,46.0,8.500000,8.500000,M,W,W,Y,1973-01-24,27.0,28.0,AAS,GSAS,APGSAS,MSE,60.0,60-75,2000.0,2001-12-21,2002.0,1,NaN,200009,5.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,2.0,3.0,0.0,0.0,17.0,17.0,2002.0,17.0,3.0,0,3.227778


In [0]:
result.to_csv("result.csv")

In [53]:
tblAwards[tblAwards['PIDM'] == 663005]

,PIDM,DownloadDte,CohortTrm,WCCID,A/C,Cons/SO,TermsElpsd,TermsAtt,CumCrAtt,CumCrErn,CumCrTran,CrAtt/Term,CrErn/Term,SEX,Eth,EthComputed,Citz,BDate,Age,GradAge,DegC,Maj,CIP,Prog,Div,ProgCredits,CreditGrp,CtlgYr,DegDte,GradAcadYr,FirstDeg,Minor,Conc,FTrmCrAtt,Seq
13931,663005,2019-09-09,200701,@00308748,A,Cons,10.0,10.0,107.0,99.0,0.0,10.700000,9.900000,M,W,X,Y,1988-02-09,15.0,19.0,AAS,ILU,500402.0,APILU,BCT,68.0,60-75,2004.0,2007-04-30,2007.0,1,NaN,NaN,200401,111111111A
47412,663005,2019-09-09,201801,@00308748,A,SO,43.0,21.0,174.0,208.0,44.0,8.285714,7.809524,M,W,X,Y,1988-02-09,15.0,30.0,AAS,NURS,513801.0,APNURS,HLT,72.0,60-75,2015.0,2018-04-30,2018.0,0,NaN,NaN,200401,111111111A.00.00.00.00.00.00.011011111C1.1A
